## Web scraping sur Jumia
## Importation des Packages

In [1]:
import requests
from random import randint
from time import sleep
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
def get_item_page(items_url):
    # télécharger la page
    response = requests.get(items_url)
    # vérifier le succès de réponse
    if response.status_code != 200:
        raise Exception('Failed to load page {}'.format(items_url))
    # Parser la réponse à l'aide de beaufifulSoup
    doc = BeautifulSoup(response.text, 'html.parser')
    return doc

In [3]:
url_jumia = 'https://www.jumia.com.tn/'

In [4]:
doc = get_item_page(url_jumia)

menu = doc.find_all('a', {'role': 'menuitem'})

## Ici je récupère les attrs href
def get_links(doc):
    links = []
    for tag in doc.findAll("a",{'role': 'menuitem'}, href=True):
        if img := tag.img:
            img = img.get("src")
        links.append(dict(url=tag.get("href"), text=tag.text, img=img)['url'])
    return links

In [93]:
# Maintenant nous avons un résumé au dessus de la fonction 
def get_item_list_categories(links):
    article_details = {
        'designation': [], 
        'prix': [],
        'image': [],
        'category': [],
        'pourcentage': []
   }
    
    for link in links:
        for num in range(20):
            items_url = f"https://www.jumia.com.tn{link}/?{num}#catalog-listing"
            response = requests.get(items_url)
            page_contents = response.text
            if response.status_code != 200:
                raise Exception('Failed to load page {}'.format(items_url))
            doc = BeautifulSoup(page_contents, "html.parser")
            for article in doc.find_all("article", {'class': "prd"}):
                if len(article.find_all('div', {'class': 'name'}, limit=1)) > 0:
                    if len(article.find_all('div', {'class': 'name'}, limit=1)) > 0:
                        article_details['designation'].append(article.find_all('div', {'class': 'name'}, limit=1)[0].text)


                    if len(article.find_all('div', {'class': 'prc'}, limit=1)) > 0:
                        article_details['prix'].append(article.find_all('div', {'class': 'prc'}, limit=1)[0].text.replace(' TND', ''))


                    if len(article.find_all('div', {'class': '_dsct'}, limit=1)) > 0:
                        article_details['pourcentage'].append(article.find_all('div', {'class': 'tag'}, limit=1)[0])


                    if len(article.find_all('img', {'class': 'img'}, limit=1)) > 0:
                        article_details['image'].append(article.find_all('img', {'class': 'img'}, limit=1)[0]['data-src'])


                    article_details['category'].append(link[1: len(link)-1])



            sleep(randint(2,4))
            print(f'Téléchargement de page {num} de category {link}' )
        
    return article_details
item_list_tags = get_item_list_categories(get_links(doc))
len (item_list_tags)

Téléchargement de page 0 de category /epicerie/
Téléchargement de page 1 de category /epicerie/
Téléchargement de page 2 de category /epicerie/
Téléchargement de page 3 de category /epicerie/
Téléchargement de page 4 de category /epicerie/
Téléchargement de page 5 de category /epicerie/
Téléchargement de page 6 de category /epicerie/
Téléchargement de page 7 de category /epicerie/
Téléchargement de page 8 de category /epicerie/
Téléchargement de page 9 de category /epicerie/
Téléchargement de page 10 de category /epicerie/
Téléchargement de page 11 de category /epicerie/
Téléchargement de page 12 de category /epicerie/
Téléchargement de page 13 de category /epicerie/
Téléchargement de page 14 de category /epicerie/
Téléchargement de page 15 de category /epicerie/
Téléchargement de page 16 de category /epicerie/
Téléchargement de page 17 de category /epicerie/
Téléchargement de page 18 de category /epicerie/
Téléchargement de page 19 de category /epicerie/
Téléchargement de page 0 de ca

5

In [108]:
for i in range(len(item_list_tags['prix'])-len(item_list_tags['pourcentage'])):
    item_list_tags['pourcentage'].append('nan')

for k,v in item_list_tags.items():
    print(f'{k} => {len(item_list_tags[k])}')
    


designation => 17160
prix => 17160
image => 17160
category => 17160
pourcentage => 17160


In [109]:
data = pd.DataFrame(item_list_tags)
data.head()

,designation,prix,image,category,pourcentage
0,Moulin d'or Pack Brownis,2.10,https://tn.jumia.is/unsafe/fit-in/300x300/filt...,epicerie,[13%]
1,Pringles Chips BBQ - 165gr,9.29,https://tn.jumia.is/unsafe/fit-in/300x300/filt...,epicerie,[13%]
2,Sicam Confiture Coing 1/2,2.35,https://tn.jumia.is/unsafe/fit-in/300x300/filt...,epicerie,[21%]
3,"Delice Punch Pomme 1,5L",2.39,https://tn.jumia.is/unsafe/fit-in/300x300/filt...,epicerie,[15%]
4,Judy Vaisselle 325 ml Citron,2.00,https://tn.jumia.is/unsafe/fit-in/300x300/filt...,epicerie,[73%]


In [110]:
data.to_csv('jumia.csv', index=False)

In [111]:
df = pd.read_csv('jumia.csv')
df.head()

,designation,prix,image,category,pourcentage
0,Moulin d'or Pack Brownis,2.10,https://tn.jumia.is/unsafe/fit-in/300x300/filt...,epicerie,"<div class=""tag _dsct"">13%</div>"
1,Pringles Chips BBQ - 165gr,9.29,https://tn.jumia.is/unsafe/fit-in/300x300/filt...,epicerie,"<div class=""tag _dsct"">13%</div>"
2,Sicam Confiture Coing 1/2,2.35,https://tn.jumia.is/unsafe/fit-in/300x300/filt...,epicerie,"<div class=""tag _dsct"">21%</div>"
3,"Delice Punch Pomme 1,5L",2.39,https://tn.jumia.is/unsafe/fit-in/300x300/filt...,epicerie,"<div class=""tag _dsct"">15%</div>"
4,Judy Vaisselle 325 ml Citron,2.00,https://tn.jumia.is/unsafe/fit-in/300x300/filt...,epicerie,"<div class=""tag _dsct"">73%</div>"
